In [1]:
import os, sys
import yaml
import inspect

In [2]:
from lxml import etree

In [3]:
from copy import deepcopy

In [4]:
from IPython.display import SVG, display

In [5]:
global SVG_EXCHANGE_OBJECT
SVG_EXCHANGE_OBJECT = None

**Helper function - SVG Line**

In [6]:
def svg_line(*args, **kwargs):
    """
    x1 = args[0]
    y1 = args[1]
    x2 = args[2]
    x3 = args[3]
    stroke = args[4]
    stroke-width args[5]
    """
    line = etree.Element("line", x1=f"{args[0]}", y1=f"{args[1]}", x2=f"{args[2]}", y2=f"{args[3]}", stroke=f"{args[4]}")
    line.set("stroke-width", f"{args[5]}")
    return line

**Initializer**

In [8]:
def init_circuit(n):
    ##SVG String  
    canvas = yaml.safe_load(open(f"{os.getcwd()}/libs/configs/quantum.circuit.params.yaml"))
    global SVG_EXCHANGE_OBJECT
    D = canvas['ygap']
    Height = D*(n+1)
    Width = canvas['xgap']
    canvas['cursor'] = deepcopy(canvas['origin'])
    SVG_EXCHANGE_OBJECT = {'n': n, 'canvas': canvas, 'svg-etree-list' : list(), 'measurement-applied' : False} 
       

**Draw Circuit Diagram**

In [9]:
def draw_svg():
    #Set Canvas Height and Width
    global SVG_EXCHANGE_OBJECT
    n = SVG_EXCHANGE_OBJECT['n']
    canvas = SVG_EXCHANGE_OBJECT['canvas']
    cursor_x, cursor_y = canvas['cursor']
    bgcolor="white"
    #######################################
    #######################################
    root = etree.Element("svg", width=f'{cursor_x+ canvas['xgap']}', height=f'{canvas['ygap']*(n+1)}', style=f"background-color: {bgcolor};", xmlns="http://www.w3.org/2000/svg")
    for ele in SVG_EXCHANGE_OBJECT['svg-etree-list']:
        #etree.tostring(etree.ElementTree(my_root_element))
        root.append(ele)
            
    svg_obj = etree.tostring(etree.ElementTree(root)).decode("ascii")
    display(SVG(data=svg_obj))
    #del SVG_EXCHANGE_OBJECT

**Qubit Wires**

In [10]:
def Wires():
    n = SVG_EXCHANGE_OBJECT['n']
    canvas = SVG_EXCHANGE_OBJECT['canvas']
    cursor_x, cursor_y = canvas['cursor']
    for i in range(n):
        x1, y1 = cursor_x, cursor_y + i*canvas['ygap']+20
        x2, y2 = cursor_x + canvas['xgap'], cursor_y + i*canvas['ygap']+20
        q_line = svg_line(x1, y1, x2, y2, "black", 2)
        SVG_EXCHANGE_OBJECT['svg-etree-list'].append(q_line)
        txt2 = etree.Element("text",  x=f"{x1-25}", y=f"{y1}", width=f"10", height=f"10")
        txt2.text =f"q_{i}"
        SVG_EXCHANGE_OBJECT['svg-etree-list'].append(txt2)
        

    
    SVG_EXCHANGE_OBJECT['canvas']['cursor'][0] = cursor_x+ canvas['xgap']

**Generic Single Qubit Gate**

In [11]:
def generic_single_gate(gates:dict):
    global SVG_EXCHANGE_OBJECT
    canvas = SVG_EXCHANGE_OBJECT['canvas']
    if not isinstance(gates, dict):
        raise TypeError("input Parameter <<gates>> must be a dictionary")

    n = SVG_EXCHANGE_OBJECT['n']
    canvas = SVG_EXCHANGE_OBJECT['canvas']
    cursor_x, cursor_y = canvas['cursor']
    L, D = canvas['xgap'], canvas['ygap']
    ######## Static Parameters #########
    rh, rw = 25, 35  
    ####################################
    for i in range(n):
        x1, y1 = cursor_x, D*(i+1)
        x2, y2 = cursor_x + L, D*(i+1)
        line =svg_line(x1, y1, x2, y2,  "black", 2)
        SVG_EXCHANGE_OBJECT['svg-etree-list'].append(line)
        if i in gates:
            bx1, by1 = x1 + (L-rw)/2, y1 - rh/2
            line1 = svg_line(cursor_x, i*D, cursor_x + (L-rw)/2, i*D ,  "black", 2)
            line2 = svg_line(cursor_x + (L+rw)/2, i*D, cursor_x + L, i*D ,  "black", 2)
            rect = etree.Element("rect", x=f"{bx1}", y=f"{by1}", width=f"{rw}", height=f"{rh}", fill="skyblue", stroke="darkblue")
            rect.set("stroke-width", "2")
            SVG_EXCHANGE_OBJECT['svg-etree-list'].append(line1)
            SVG_EXCHANGE_OBJECT['svg-etree-list'].append(rect)
            SVG_EXCHANGE_OBJECT['svg-etree-list'].append(line2)
            txt1 = etree.Element("text",  x=f"{bx1+8}", y=f"{by1+15}", width=f"10", height=f"10")
            txt1.text =f"{gates[i]}"
            txt1.set("fill", "black")
            SVG_EXCHANGE_OBJECT['svg-etree-list'].append(txt1)

        
    SVG_EXCHANGE_OBJECT['canvas']['cursor'][0] = cursor_x +L

**Generic Multi Qubit Gate**

In [12]:
def generic_multi_gate(from_qubit, to_qubit, gate_label=None):
    """
    #TBA
    """
    global SVG_EXCHANGE_OBJECT
    canvas = SVG_EXCHANGE_OBJECT['canvas']
    n = SVG_EXCHANGE_OBJECT['n']
    canvas = SVG_EXCHANGE_OBJECT['canvas']
    cursor_x, cursor_y = canvas['cursor']
    L, D = canvas['xgap'], canvas['ygap']
    #############
    condition1 = from_qubit < to_qubit
    condition2 = (to_qubit - from_qubit) >1
    condition3 = from_qubit < to_qubit <= n
    if not all([condition1, condition2, condition3]):
        raise Exception("TBA")
    
    ######## Static Parameters #########
    rh, rw = 25, 35  
    ####################################
    for i in range(n):
        #if not i in range(from_qubit, to_qubit):
        line1 = svg_line(cursor_x, i*D, cursor_x + L, i*D ,  "black", 2)
        SVG_EXCHANGE_OBJECT['svg-etree-list'].append(line1)
    
    
    for j in range(from_qubit, to_qubit):
        line_1 = svg_line(cursor_x, j*D, cursor_x + (L-rw)/2, j*D ,  "black", 2)
        SVG_EXCHANGE_OBJECT['svg-etree-list'].append(line_1)
        line_2 = svg_line(cursor_x + (L+rw)/2, j*D, cursor_x + L, j*D ,  "black", 2)
        SVG_EXCHANGE_OBJECT['svg-etree-list'].append(line_2)

    delta = to_qubit - from_qubit
    bw = L - rw/2
    bh = 2*rh + delta*D
    bx1= cursor_x + (L-rw)/2 
    by1 = cursor_y
    ######   Box Design #####
    rect = etree.Element("rect", x=f"{bx1}", y=f"{by1}", width=f"{bw}", height=f"{bh}", fill="skyblue", stroke="darkblue")
    rect.set("stroke-width", "2")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(rect)
    #####  Add text  ######
    if not gate_label:
        gate_label= "Multi Qubit Gate"

    text_len = len(gate_label)
    txt_bx = cursor_x + (L)/2
    txt_by = cursor_y + delta*(D-text_len)/2
    text_label = etree.Element("text",  x=f"{txt_bx}", y=f"{txt_by}", width=f"{bw}", height=f"{bh}")
    text_label.set("fill", "black")
    text_label.set("transform", f"rotate(90, {txt_bx}, {txt_by})")
    text_label.text = gate_label
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(text_label)
    ######### Finally #####################
    SVG_EXCHANGE_OBJECT['canvas']['cursor'][0] = cursor_x+L

**Controlled NOT/X Gate**

In [13]:
def cnot_gate(control:int, target:int):
    global SVG_EXCHANGE_OBJECT
    n = SVG_EXCHANGE_OBJECT['n']
    canvas = SVG_EXCHANGE_OBJECT['canvas']
    L = canvas['xgap']  # Y Axis Gap
    D = canvas['ygap']  # X Axis Gap
    #Set Canvas Height and Width
    condition1 = not control == target
    condition2 = control < n and target <n
    if not(condition1 and condition2):
        raise Exception("CNOT Gate cannot operate on same/one qubit")

    cursor_x, cursor_y = canvas['cursor']
    y_buffer = 20
    cr, Cr = 3, 15
    circ_x_adjust_by = 8
    for i in range(n):
        if not i==target:
            q_line = svg_line(cursor_x, cursor_y +y_buffer +i*D, cursor_x+L, cursor_y +y_buffer + i*D, "black", 2)
            SVG_EXCHANGE_OBJECT['svg-etree-list'].append(q_line)


    delta = abs(control - target)
    ##########  Anchor(control) Circle   ##########
    cx1 = cursor_x + (L-Cr)/2
    cy1 = cursor_y + y_buffer + control*D
    control_anchor = etree.Element("circle", r= f"{cr}",cx=f"{cx1}", cy=f"{cy1}", fill="black", stroke="black")
    control_anchor.set("stroke-width", "2")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(control_anchor)
    ##########  Anchor(control) Full Horizontal line segment   ##########
    hline_ax1_1, hline_ay1_1 = cursor_x -(L-Cr)/2, cursor_y + target*D + y_buffer
    hline_ax2_1, hline_ay2_1 = cursor_x+(L-Cr)/2, cursor_y + target*D + y_buffer
    hline_ctrl1 = etree.Element("line", x1=f"{hline_ax1_1}",  y1=f"{hline_ay1_1}", x2 = f"{hline_ax2_1}", y2 = f"{hline_ay2_1}", stroke = "black")
    hline_ctrl1.set('stroke-width' , "2")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(hline_ctrl1)
    ###########################################################
    ##########  Anchor(control) and Target Connecting Vertical line segment   ##########
    vline_x11 =  cursor_x + (L-Cr)/2
    vline_y11 = cursor_y + control*D + y_buffer
    vline_x21 = cursor_x + (L-Cr)/2
    vline_y21 = cursor_y  + target*D + y_buffer 
    vline_ct = etree.Element("line", x1=f"{vline_x11}",  y1=f"{vline_y11}", x2 = f"{vline_x21}", y2 = f"{vline_y21}", stroke = "black")
    vline_ct.set('stroke-width' , "2")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(vline_ct)
    ###########################################################
    hline_tgt2_x1, hline_tgt2_y1 = cursor_x + (L - 3*Cr)/2, cursor_y+  target*D +y_buffer
    hline_tgt2_x2, hline_tgt2_y2 = cursor_x + L, cursor_y + target*D + y_buffer
    hline_tgt22 = etree.Element("line", x1=f"{hline_tgt2_x1}",  y1=f"{hline_tgt2_y1}", x2 = f"{hline_tgt2_x2}", y2 = f"{hline_tgt2_y2}", stroke = "black") 
    hline_tgt22.set('stroke-width', "2")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(hline_tgt22)
    ###########################################################
    ############ Target Anchor(Circle) ########################
    cx2 = cursor_x+ L/2 - circ_x_adjust_by
    cy2 = cursor_y + y_buffer + target*D
    target_anchor = etree.Element("circle",r = f"{Cr}",cx = f"{cx2}",cy =  f"{cy2}", fill= "skyblue", stroke="black")
    target_anchor.set('stroke-width', "2")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(target_anchor)
    ###########################################################
    ################ Text Label       #########################
    txt1_ele = etree.Element("text", x= f"{cx2-5}", y= f"{cy2+5}", style="color:darkblue;bold;")
    txt1_ele.set('font-size',  "16")
    txt1_ele.text ="X"
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(txt1_ele)
    #####  Finally  #########
    SVG_EXCHANGE_OBJECT['canvas']['cursor'][0] +=L

**Controlled Z Gate**

In [14]:
def cz_gate(control, target):
    global SVG_EXCHANGE_OBJECT
    n = SVG_EXCHANGE_OBJECT['n']
    canvas = SVG_EXCHANGE_OBJECT['canvas']
    L = canvas['xgap']  # Y Axis Gap
    D = canvas['ygap']  # X Axis Gap
    cursor_x, cursor_y = canvas['cursor']
    cr, y_buffer = 3, 20
    x_adjust_by = 1
    
    if control == target:
        raise Exception("CNOT Gate cannot operate on same/one qubit")
    #####-------------------------------------------------------------------------------------------#####

    for i in range(n):
        q_line = svg_line(cursor_x, cursor_y +y_buffer +i*D, cursor_x+L, cursor_y +y_buffer + i*D, "black", 2)
        SVG_EXCHANGE_OBJECT['svg-etree-list'].append(q_line)

    delta = abs(control-target)
    ######### Control to Target Linking Vline ###########
    vline_x11 =  cursor_x + (L-cr)/2 + x_adjust_by
    vline_y11 = cursor_y + control*D + y_buffer
    vline_x21 = cursor_x + (L-cr)/2 + x_adjust_by
    vline_y21 = cursor_y  + target*D + y_buffer 
    vline_ct = etree.Element("line", x1=f"{vline_x11}",  y1=f"{vline_y11}", x2 = f"{vline_x21}", y2 = f"{vline_y21}", stroke = "black")
    vline_ct.set('stroke-width' , "2")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(vline_ct)
    
    ########### Control End   ##################
    cx1 = cursor_x+ L/2 
    cy1 = cursor_y + y_buffer + control*D
    control_anchor = etree.Element("circle",r = f"{cr}",cx = f"{cx1}",cy =  f"{cy1}", fill= "skyblue", stroke="black")
    control_anchor.set('stroke-width', "2")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(control_anchor)
    ########### Target End    ##################
    cx2 = cursor_x+ L/2 
    cy2 = cursor_y + y_buffer + target*D
    target_anchor = etree.Element("circle",r = f"{cr}",cx = f"{cx2}",cy =  f"{cy2}", fill= "skyblue", stroke="black")
    target_anchor.set('stroke-width', "2")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(target_anchor)
    #############################################
    txt_bx = cursor_x + (L)/2
    txt_by = control*(cursor_y ) + target*(D - 7)/2 + y_buffer
    text_label = etree.Element("text",  x=f"{txt_bx}", y=f"{txt_by}", width=f"{txt_bx}", height=f"{txt_by}")
    text_label.set("fill", "brown")
    text_label.set("transform", f"rotate(90, {txt_bx}, {txt_by})")
    text_label.text = "CZ Gate"
    text_label.set("style", "font-weight: bold;")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(text_label)
    ######## Finally #########
    SVG_EXCHANGE_OBJECT['canvas']['cursor'][0] +=L



**SWAP Gate**

In [15]:
def swap_gate(control, target):
    """
    TBA
    """
    global SVG_EXCHANGE_OBJECT
    n = SVG_EXCHANGE_OBJECT['n']
    canvas = SVG_EXCHANGE_OBJECT['canvas']
    L = canvas['xgap']  # Y Axis Gap
    D = canvas['ygap']  # X Axis Gap
    cursor_x, cursor_y = canvas['cursor']
    #Set Canvas Height and Width
    y_buffer = 20
    x_adjust_by = 1
    condition1 = not control == target
    condition2 = control < n and target <n
    if not(condition1 and condition2):
        raise Exception("CNOT Gate cannot operate on same/one qubit")

    for i in range(n):
        q_line = svg_line(cursor_x, cursor_y +y_buffer +i*D, cursor_x+L, cursor_y +y_buffer + i*D, "black", 2)
        SVG_EXCHANGE_OBJECT['svg-etree-list'].append(q_line)


    
    x_adjust_by, y_buffer  =1, 20
    delta = abs(control-target)
    ######### Control to Target Linking Vline ###########
    vline_x11 =  cursor_x + L/2 + x_adjust_by
    vline_y11 = cursor_y + control*D + y_buffer
    vline_x21 = cursor_x + L/2 + x_adjust_by
    vline_y21 = cursor_y  + target*D + y_buffer 
    vline_ct = etree.Element("line", x1=f"{vline_x11}",  y1=f"{vline_y11}", x2 = f"{vline_x21}", y2 = f"{vline_y21}", stroke = "black")
    vline_ct.set('stroke-width' , "2")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(vline_ct)
    ####Static param  ####
    XL=15
    x11, y11 = cursor_x + L/2 -5, cursor_y + y_buffer + control*D +5
    text_label1 = etree.Element("text",  x=f"{x11}", y=f"{y11}", width="50", height="50")
    text_label1.set("fill", "darkred")
    text_label1.text = "X"
    text_label1.set("style", "font-weight: bold;")
    text_label1.set("stroke", "darkblue")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(text_label1)
    x21, y21 = cursor_x + L/2-5, cursor_y + +y_buffer +target*D + 5
    text_label2 = etree.Element("text",  x=f"{x21}", y=f"{y21}", width="50", height="50")
    text_label2.set("fill", "darkred")
    text_label2.text = "X"
    text_label2.set("style", "font-weight: bold;")
    text_label2.set("stroke", "darkblue")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(text_label2)  
    ######## Finally #########
    SVG_EXCHANGE_OBJECT['canvas']['cursor'][0] +=L


In [2]:
def measurement_gate():
    """
    TBA
    """
    global SVG_EXCHANGE_OBJECT
    n = SVG_EXCHANGE_OBJECT['n']
    canvas = SVG_EXCHANGE_OBJECT['canvas']
    L = canvas['xgap']  # Y Axis Gap
    D = canvas['ygap']  # X Axis Gap
    cursor_x, cursor_y = canvas['cursor']
    #Set Canvas Height and Width
    y_buffer = 20
    x_adjust_by = 1

    for i in range(n):
        q_line = svg_line(cursor_x, cursor_y + y_buffer +i*D, cursor_x+L, cursor_y +y_buffer + i*D, "black", 2)
        SVG_EXCHANGE_OBJECT['svg-etree-list'].append(q_line)
        circles = etree.Element("circle",r = f"5",cx = f"{cursor_x + (L-5)/2}",cy =  f"{cursor_y +y_buffer+i*(D)}", fill= "black", stroke="red")
        SVG_EXCHANGE_OBJECT['svg-etree-list'].append(circles)
        
    
    polygon = etree.Element("polygon", style="fill:skyblue;stroke:black;stroke-width:2;")
    p0, p1 = cursor_x + (L-5)/2, cursor_y - D/4
    p2, p3 = cursor_x + (L-5)/2, cursor_y + D*(n - 1/4)
    p4, p5 = cursor_x + L, cursor_y + D*(n - 1/4)
    p6, p7 = cursor_x + L, cursor_y - D/4
    p8, p9 = cursor_x + 1.5*L, cursor_y + n*D/2
    polygon.set("points", f"{p0},{p1} {p2},{p3} {p4},{p5} {p6},{p7}")
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(polygon)
    ###############################################
    txt_bx = cursor_x + L/2+15
    txt_by = cursor_y + (abs(p1-p5)-11)/4
    text_label = etree.Element("text",  x=f"{txt_bx}", y=f"{txt_by}")
    text_label.set("fill", "black")
    text_label.set("font-weight", "bold")
    text_label.set("transform", f"rotate(90, {txt_bx}, {txt_by})")
    text_label.text = "Measurement"
    SVG_EXCHANGE_OBJECT['svg-etree-list'].append(text_label) 
    ################################################
    for x_ in range(n):
        box = etree.Element("rect",   x=f"{cursor_x + x_*15}", y=f"{cursor_y + n*D}", width="15", height="15", fill="none",stroke="black")
        box.set('stroke-width', "1")
        SVG_EXCHANGE_OBJECT['svg-etree-list'].append(box)

        
    ######## Finally #########
    SVG_EXCHANGE_OBJECT['canvas']['cursor'][0] +=L